In [21]:
# KT Noted: Using DeepLearning environment
# KT Noted: ChatGPT o3-mini-high (6 Feb 2025)

In [22]:
import random
import pandas as pd
import numpy as np

In [23]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch

In [25]:
# Set seeds for reproducibility
random.seed(42)              # For Python's built-in random module
np.random.seed(42)           # For NumPy
tf.random.set_seed(42)       # For TensorFlow

In [26]:
# -------------------------------
# 1. Data Preparation & Splitting
# -------------------------------

In [27]:
# Load the CSV file
file_path = r"C:\Users\kornn\Dropbox\00_EnergyModel_Weather\Dataset\Training_Combinded\train_df.csv"  
train_df = pd.read_csv(file_path)
train_df.head()

,Date,Total_kWh,Hour_of_Day,Day_of_week,Holiday,Weekend,Weekday,Working_Hour,Noon,Temperature,Dew Point,Humidity,Pressure,Solar
0,2018-07-01 00:00:00,176.261667,0,6,0,1,0,0,0,25.0,24.2,95,1006.43,0
1,2018-07-01 01:00:00,177.300000,1,6,0,1,0,0,0,25.5,24.8,96,1006.43,0
2,2018-07-01 02:00:00,177.048833,2,6,0,1,0,0,0,25.8,24.9,95,1005.76,0
3,2018-07-01 03:00:00,175.129167,3,6,0,1,0,0,0,25.7,24.7,94,1005.08,0
4,2018-07-01 04:00:00,183.175000,4,6,0,1,0,0,0,26.3,24.6,90,1005.42,0


In [28]:
train_df['Date'] = pd.to_datetime(train_df['Date']) # Make Date column is in datetime format

In [29]:
# Train and Test Split 
train_set = (train_df['Date'] >= '2018-07-01 00:00:00') & (train_df['Date'] <= '2019-06-30 23:00:00')
test_set = (train_df['Date'] >= '2019-07-01 00:00:00') & (train_df['Date'] <= '2019-08-31 23:00:00')

In [30]:
# Create the training and test DataFrames
train_data = train_df.loc[train_set].copy()
test_data = train_df.loc[test_set].copy()

In [31]:
train_data

,Date,Total_kWh,Hour_of_Day,Day_of_week,Holiday,Weekend,Weekday,Working_Hour,Noon,Temperature,Dew Point,Humidity,Pressure,Solar
0,2018-07-01 00:00:00,176.261667,0,6,0,1,0,0,0,25.0,24.2,95,1006.43,0
1,2018-07-01 01:00:00,177.300000,1,6,0,1,0,0,0,25.5,24.8,96,1006.43,0
2,2018-07-01 02:00:00,177.048833,2,6,0,1,0,0,0,25.8,24.9,95,1005.76,0
3,2018-07-01 03:00:00,175.129167,3,6,0,1,0,0,0,25.7,24.7,94,1005.08,0
4,2018-07-01 04:00:00,183.175000,4,6,0,1,0,0,0,26.3,24.6,90,1005.42,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2019-06-30 19:00:00,154.142667,19,6,0,1,0,0,0,30.0,25.5,77,1004.00,0
8756,2019-06-30 20:00:00,155.219167,20,6,0,1,0,0,0,29.8,25.6,78,1005.00,0
8757,2019-06-30 21:00:00,129.356167,21,6,0,1,0,0,0,29.3,26.2,83,1005.00,0
8758,2019-06-30 22:00:00,128.086167,22,6,0,1,0,0,0,29.1,26.3,85,1006.00,0


In [32]:
# ---------------------------------------
# 2. Set Features, Target and Preprocess
# ---------------------------------------

In [33]:
target_col = 'Total_kWh' # Set the target variable
feature_cols = [col for col in train_df.columns if col not in ['Date', target_col]] # Set the features

In [34]:
# Data Preparation for LSTM and Keras (Numpy Array)
X_train = train_data[feature_cols].values    # Converts df to a NumPy array.
y_train = train_data[target_col].values.reshape(-1, 1) # The target is 1D array, but Keras models expect the target to be in a 2D shape.
X_test = test_data[feature_cols].values
y_test = test_data[target_col].values.reshape(-1, 1)

In [35]:
# Scaling to ensures that all features contribute equally to the model training.
# LSTM models are sensitive to the scale of input features.
scaler_X = MinMaxScaler() # between 0 and 1
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)   # Transform using the same scaler used for training set, as do not see test set.
y_test_scaled = scaler_y.transform(y_test)

In [36]:
# Reshape input to be 3D [samples, timesteps, features] as required by LSTM
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [37]:
# -------------------------------
# 3. Model Training LSTM
# -------------------------------

In [38]:
def build_model(hp):
    model = Sequential()
    
    # First Bidirectional LSTM layer: returns sequences for the next layer
    model.add(Bidirectional(LSTM(
        units=hp.Int('units_0', min_value=32, max_value=256, step=32),
        return_sequences=True,
        recurrent_dropout=hp.Float('recurrent_dropout_0', min_value=0.0, max_value=0.5, step=0.1)),
        input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
    
    model.add(Dropout(hp.Float('dropout_0', min_value=0.1, max_value=0.5, step=0.1)))
    
    # Second Bidirectional LSTM layer: final LSTM layer, so return_sequences is False
    model.add(Bidirectional(LSTM(
        units=hp.Int('units_1', min_value=32, max_value=256, step=32),
        return_sequences=False,
        recurrent_dropout=hp.Float('recurrent_dropout_1', min_value=0.0, max_value=0.5, step=0.1))))
    
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
    
    # Final Output Layer:
    # The Dense layer here is fully connected by default in Keras.
    model.add(Dense(1))  
    
    # Compile the model with a tunable learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate',
                      min_value=1e-4, max_value=1e-2, 
                      sampling='LOG')),
        loss='mse')
    
    return model

In [39]:
# Hyperparameter Tuning Setup using RandomSearch
random_search = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=30,           # Exploring search space
    executions_per_trial=2,
    directory='LSTM_Tuning_Logs',
    project_name='LSTM_Tuning_Model_02')

In [40]:
# Train the model 
random_search.search(
    X_train_scaled, 
    y_train_scaled, 
    epochs=100, 
    validation_split=0.2, 
    shuffle=False,            # Ensures reproducibility 
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

Trial 30 Complete [00h 04m 53s]
val_loss: 0.009156211744993925

Best val_loss So Far: 0.00556799815967679
Total elapsed time: 01h 37m 03s


In [41]:
# Retrieve and review the best model
best_model_params = random_search.get_best_models(num_models=1)[0]
best_model_params.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 1, 192)            83712     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 1, 192)            0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               131584    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 215425 (841.50 KB)
Trainable params: 21542

In [42]:
# -------------------------------
# 4. Result on Train set 
# -------------------------------

In [43]:
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    cv_rmse = (rmse / np.mean(y_true)) * 100
    nmbe = (np.mean(y_pred - y_true) / np.mean(y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    return {'rmse': rmse, 'cv_rmse': cv_rmse, 'nmbe': nmbe, 'r2': r2}

In [44]:
# Predict on scaled training data
y_pred_train_scaled = best_model_params.predict(X_train_scaled)

y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled) # Inverse transform predictions and true values back to original scale
train_metrics = evaluate_model(y_train, y_pred_train)

274/274 [==============================] - 1s 3ms/step


In [45]:
print("Evaluation Metrics on Train Set (Random Search):")
print("RMSE: {:.3f}".format(train_metrics['rmse']))
print("CV(RMSE): {:.2f}%".format(train_metrics['cv_rmse']), "| IPMVP < 20% | ASHRAE & FEMP < 30%")
print("NMBE: {:.2f}%".format(train_metrics['nmbe']), "| IPMVP < 5% | ASHRAE & FEMP < 10%")
print("R2 Score: {:.3f}".format(train_metrics['r2']), "| IPMVP & ASHRAE > 0.75 | FEMP = NA")

Evaluation Metrics on Train Set (Random Search):
RMSE: 107.131
CV(RMSE): 41.36% | IPMVP < 20% | ASHRAE & FEMP < 30%
NMBE: 13.52% | IPMVP < 5% | ASHRAE & FEMP < 10%
R2 Score: 0.712 | IPMVP & ASHRAE > 0.75 | FEMP = NA


In [46]:
# -------------------------------
# 4. Result on Test set 
# -------------------------------

In [47]:
# Model Evaluation
y_pred_test_scaled = best_model_params.predict(X_test_scaled)

y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled) # Inverse transform predictions and true values back to original scale
test_metrics = evaluate_model(y_test, y_pred_test)

47/47 [==============================] - 0s 3ms/step


In [48]:
print("Evaluation Metrics on Test Set (Random Search):")
print("RMSE: {:.3f}".format(test_metrics['rmse']))
print("CV(RMSE): {:.2f}%".format(test_metrics['cv_rmse']), "| IPMVP < 20% | ASHRAE & FEMP < 30%")
print("NMBE: {:.2f}%".format(test_metrics['nmbe']), "| IPMVP < 5% | ASHRAE & FEMP < 10%")
print("R2 Score: {:.3f}".format(test_metrics['r2']), "| IPMVP & ASHRAE > 0.75 | FEMP = NA")

Evaluation Metrics on Test Set (Random Search):
RMSE: 73.067
CV(RMSE): 25.51% | IPMVP < 20% | ASHRAE & FEMP < 30%
NMBE: 3.41% | IPMVP < 5% | ASHRAE & FEMP < 10%
R2 Score: 0.904 | IPMVP & ASHRAE > 0.75 | FEMP = NA


In [ ]:
##################### The End #####################